In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Concatenate
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import os
import glob


In [ ]:
# read from csv and convert to dataframe
path="files/data.csv"
df = pd.read_csv(path)
df.pop("id")
df.head()

In [ ]:
# Define the directory path where your invoice images are located
directory_path = path  # Replace with the actual directory path

# Define a list of image file extensions (add more if needed)
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif', '*.bmp']

# Initialize lists to store image data and labels
image_data = []  # Store your image data here
labels = []  # Store corresponding recipient names here

# Load and preprocess images
for extension in image_extensions:
    pattern = os.path.join(directory_path, extension)
    image_files = glob.glob(pattern)
    for img_path in image_files:
        img = plt.imread(img_path)
        img = tf.image.resize(img, (256, 256))  # Resize the image if needed
        img = img / 255.0  # Normalize pixel values to [0, 1]
        image_data.append(img)

        # Assuming you have a way to determine recipient names for each image, append them to 'labels' list.
        # Example: labels.append("Recipient Name") for each image.
        # Make sure the number of labels matches the number of images.
        labels.append(df)  # Replace with the actual recipient name.



In [20]:

# Convert lists to TensorFlow tensors
image_tensors = tf.convert_to_tensor(image_data)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize recipient names using BERT tokenizer
recipient_name_tokens = [tokenizer.encode(name, add_special_tokens=True, max_length=128, truncation=True) for name in labels]

# Pad tokenized sequences to a consistent length (e.g., 128 tokens)
max_length = 128
recipient_name_tokens = [tokens + [0] * (max_length - len(tokens)) for tokens in recipient_name_tokens]

# Convert tokenized recipient names to TensorFlow tensors
recipient_name_tensors = tf.convert_to_tensor(recipient_name_tokens)



KeyboardInterrupt: 

In [ ]:
# Define the image analysis model
image_input = Input(shape=(256, 256, 3))
image_features = YourImageAnalysisModel(image_input)  # Replace with your image analysis model
image_features = Dense(128, activation='relu')(image_features)

# Define the BERT model for text processing
text_input = Input(shape=(max_length,))
text_features = bert_model(text_input)[0][:, 0, :]  # Use the [CLS] token embedding

# Concatenate image and text features
combined_features = Concatenate()([image_features, text_features])

# Define the output layer to predict recipient names
output = Dense(len(labels), activation='softmax')(combined_features)

# Create the combined model
combined_model = tf.keras.Model(inputs=[image_input, text_input], outputs=output)

# Compile the model with an appropriate loss and optimizer
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using image_tensors and recipient_name_tensors
combined_model.fit([image_tensors, recipient_name_tensors], labels, epochs=10)

# Save the model
combined_model.save('combined_model.h5')